In [17]:
from rk4step import rk4step
import numpy as np
import casadi as ca

In [19]:
Ts = .5
N = 19
x0 = np.array([0])
t_grid = np.linspace(0, Ts*N, N+1)
t_span = [0, Ts*N]
t0 = 0

In [21]:
k = ca.MX.sym('k',1)
x = ca.MX.sym('x',2)
u = ca.MX.sym('u',1)
w = ca.MX.sym('w',1)

k = ca.MX.sym('k',1)
x = ca.MX.sym('x',2)
u = ca.MX.sym('u',1)
w = ca.MX.sym('w',1)

In [16]:
P_e = 10
P_i = 5
C = 50
M = 5

def V(w):
    return w * C 

def O_i(k, O_i_prev, u):
    return O_i_prev + np.sign(u) * M * k

def O_e(k, w, O_i_prev, u):
    return ca.max(V(w) - O_i(k, O_i_prev, u), 0)

def L_e(k, w, x, u):
    return P_e * O_e(k, w, x[0], u)

def L_i(k, x, u):
    P_i * O_i(k, x[0], u)

#L_e = ca.Function('L_e', [k, w, x, u], [L_e])


In [20]:
X_rk4 = np.zeros((x0.size, N+1))
X_rk4[:,0] = x0
for k in range(1, len(t_grid)):
    X_rk4[:,k] = rk4step(L_e, Ts,  X_rk4[:,k-1], t0).full().flatten()

TypeError: L_e() missing 2 required positional arguments: 'x' and 'u'

In [13]:
X_rk4[:,-1]

array([-0.38768787,  0.00873917])